In [62]:
import pandas as pd
from datetime import timedelta, date
import re
import requests
from bs4 import BeautifulSoup

In [115]:
## Getting the web page and content
page = requests.get('https://weather.com/weather/tenday/l/San+Francisco+CA?canonicalCityId=dfdaba8cbe3a4d12a8796e1f7b1ccc7174b4b0a2d5ddb1c8566ae9f154fa638c')
print(page)
soup = BeautifulSoup(page.content,'html.parser')

<Response [200]>


In [117]:
#Scrapping the first 6 forcasted days
day_forecast = soup.find_all('span',class_="DailyContent--daypartDate--2A3Wi")[0:12]
days = []
for i in day_forecast:
    days.append(i.text)
days


['Mon 14',
 'Mon 14',
 'Tue 15',
 'Tue 15',
 'Wed 16',
 'Wed 16',
 'Thu 17',
 'Thu 17',
 'Fri 18',
 'Fri 18',
 'Sat 19',
 'Sat 19']

In [119]:
#Generating the dates for the 6days
today = date.today()
yesterday = today - timedelta(days = 1)
dates_forecast =pd.date_range(start= yesterday, periods=6)

#Creating a list of repeated dates
repeated_dates = []
for date1, date2 in zip(dates_forecast, dates_forecast):
    repeated_dates.append(date1.date())
    repeated_dates.append(date2.date())
repeated_dates = repeated_dates[ 0:len(repeated_dates)]
repeated_dates

[datetime.date(2022, 3, 14),
 datetime.date(2022, 3, 14),
 datetime.date(2022, 3, 15),
 datetime.date(2022, 3, 15),
 datetime.date(2022, 3, 16),
 datetime.date(2022, 3, 16),
 datetime.date(2022, 3, 17),
 datetime.date(2022, 3, 17),
 datetime.date(2022, 3, 18),
 datetime.date(2022, 3, 18),
 datetime.date(2022, 3, 19),
 datetime.date(2022, 3, 19)]

In [113]:
#Scrapping forecasted description for the 6days
forecast_decription = soup.find_all('p',class_="DailyContent--narrative--hplRl")[0:12]
description=[]
for t in forecast_decription:
    description.append(t.text)

description

['A mix of clouds and sun. High 66F. Winds WSW at 10 to 15 mph.',
 'Rain showers early becoming a steady light rain overnight. Low 53F. Winds SSW at 5 to 10 mph. Chance of rain 70%.',
 'Showers in the morning, then cloudy in the afternoon. High around 60F. Winds WSW at 10 to 15 mph. Chance of rain 50%.',
 'Partly cloudy. Low 47F. Winds W at 10 to 15 mph.',
 'Partly cloudy skies. High 62F. Winds W at 10 to 20 mph.',
 'Partly cloudy skies during the evening will give way to cloudy skies overnight. Low 48F. Winds W at 10 to 15 mph.',
 'Cloudy skies early will become partly cloudy later in the day. High near 60F. Winds WSW at 10 to 20 mph.',
 'Mainly clear early, then a few clouds later on. Low 47F. Winds W at 10 to 15 mph.',
 'Sunshine and clouds mixed. High near 60F. Winds WSW at 10 to 20 mph.',
 'Cloudy with occasional showers late at night. Low 48F. Winds WSW at 10 to 15 mph. Chance of rain 50%.',
 'Rain likely. Winds increasing late. High 56F. Winds WSW at 20 to 30 mph. Chance of rain

In [102]:
#Scrapping forecasted temperature for the 6days
daily_temp=[]
for t in forecast_decription:
    daily_temp.append(re.findall('[0-9]+',(t.text.split('F')[0]))[0:12])
daily_temp = [temp for sublist in daily_temp for temp in sublist]
daily_temp = [int(i) for i in daily_temp]
Converted_temp = [round((fahri-32)*(5/9),2) for fahri in daily_temp]
Converted_temp

[18.89, 11.67, 15.56, 8.33, 16.67, 8.89, 15.56, 8.33, 15.56, 8.89, 13.33, 6.67]

In [120]:
##Creating a dataframe for the scrapped forcasts
final_data =pd.DataFrame({'Days':days, 'Date':repeated_dates,'Description':description,'Temperature in Celsius':Converted_temp}) 
final_data

,Days,Date,Description,Temperature in Celsius
0,Mon 14,2022-03-14,A mix of clouds and sun. High 66F. Winds WSW a...,18.89
1,Mon 14,2022-03-14,Rain showers early becoming a steady light rai...,11.67
2,Tue 15,2022-03-15,"Showers in the morning, then cloudy in the aft...",15.56
3,Tue 15,2022-03-15,Partly cloudy. Low 47F. Winds W at 10 to 15 mph.,8.33
4,Wed 16,2022-03-16,Partly cloudy skies. High 62F. Winds W at 10 t...,16.67
5,Wed 16,2022-03-16,Partly cloudy skies during the evening will gi...,8.89
6,Thu 17,2022-03-17,Cloudy skies early will become partly cloudy l...,15.56
7,Thu 17,2022-03-17,"Mainly clear early, then a few clouds later on...",8.33
8,Fri 18,2022-03-18,Sunshine and clouds mixed. High near 60F. Wind...,15.56
9,Fri 18,2022-03-18,Cloudy with occasional showers late at night. ...,8.89
